In [4]:
import torch
import numpy as np
import torchvision
import pandas as pd
from PIL import Image
import torch.nn as nn
from pathlib import Path
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

# Read csv

In [5]:
#read the csv
#column Breed shows the y with the image name
data_csv = pd.read_csv('dataset_CV.csv')
data_csv

,DogId,Breed,Image,ImageNumber
0,9473,AMERICAN STAFFORDSHIRE BULL TERRIER,89f3c062-8928-487b-b7c0-8ab50f44641e.jpg,1
1,9473,AMERICAN STAFFORDSHIRE BULL TERRIER,e9e33b00-5ec4-4742-ac02-b9ee2a88314f.jpg,2
2,9473,AMERICAN STAFFORDSHIRE BULL TERRIER,2a81e50c-efdb-4e98-9220-d6945753731d.jpg,3
3,9473,AMERICAN STAFFORDSHIRE BULL TERRIER,0609e883-2c18-40b2-a4a5-8a90754a94f6.jpg,4
4,9548,AMERICAN STAFFORDSHIRE BULL TERRIER,10fb3092-842a-466b-a3db-3c88f624c6f4.jpg,1
...,...,...,...,...
4139,38513,BORDER COLLIE,e3957a7c-9289-4441-bdbd-d2289bef745d.jpg,5
4140,38513,BORDER COLLIE,48afd0ab-baf2-4ee8-9584-9563cec529e7.jpg,6
4141,38513,BORDER COLLIE,8f8c4ea0-7426-4188-bb6c-94fa1ecb1e33.jpg,7
4142,38513,BORDER COLLIE,cce5499f-eb04-4d8c-8d3f-c9c9bee16c5c.jpg,8


# Preprocessing

In [24]:
# setting transforms to use on images
trans = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=0.5, std=0.5),
    transforms.Resize((80, 80)),
    ])

In [25]:
location = r"dog_breed_photos"

# prepare the image by setting to RGB and applying the transforms (return tensor)
def create_data(path):
    image = Image.open(Path(location) / path).convert('RGB')
    img = trans(image)
    image.close()
    return img

# prepare the label and turn to tensor
def create_label(label):
    if label == "AMERICAN STAFFORDSHIRE BULL TERRIER":
        y = 0
    elif label == "BULL ARAB":
        y = 1
    elif label == "GERMAN SHEPHERD":
        y = 2
    elif label == "SIBERIAN HUSKY":
        y = 3
    elif label == "BORDER COLLIE":
        y = 4
    return torch.tensor(y)
    

data_csv['data'] = data_csv['Image'].apply(lambda image: create_data(image))
data_csv['class'] = data_csv['Breed'].apply(lambda breed: create_label(breed))
data_csv

,DogId,Breed,Image,ImageNumber,data,class
0,9473,AMERICAN STAFFORDSHIRE BULL TERRIER,89f3c062-8928-487b-b7c0-8ab50f44641e.jpg,1,"[[[tensor(-0.0431), tensor(0.0419), tensor(0.0...",tensor(0)
1,9473,AMERICAN STAFFORDSHIRE BULL TERRIER,e9e33b00-5ec4-4742-ac02-b9ee2a88314f.jpg,2,"[[[tensor(1.), tensor(1.), tensor(1.), tensor(...",tensor(0)
2,9473,AMERICAN STAFFORDSHIRE BULL TERRIER,2a81e50c-efdb-4e98-9220-d6945753731d.jpg,3,"[[[tensor(1.), tensor(1.), tensor(1.), tensor(...",tensor(0)
3,9473,AMERICAN STAFFORDSHIRE BULL TERRIER,0609e883-2c18-40b2-a4a5-8a90754a94f6.jpg,4,"[[[tensor(1.), tensor(1.), tensor(1.), tensor(...",tensor(0)
4,9548,AMERICAN STAFFORDSHIRE BULL TERRIER,10fb3092-842a-466b-a3db-3c88f624c6f4.jpg,1,"[[[tensor(0.3373), tensor(0.3567), tensor(0.36...",tensor(0)
...,...,...,...,...,...,...
4139,38513,BORDER COLLIE,e3957a7c-9289-4441-bdbd-d2289bef745d.jpg,5,"[[[tensor(-0.7552), tensor(-0.7529), tensor(-0...",tensor(4)
4140,38513,BORDER COLLIE,48afd0ab-baf2-4ee8-9584-9563cec529e7.jpg,6,"[[[tensor(-0.9216), tensor(-0.8598), tensor(-0...",tensor(4)
4141,38513,BORDER COLLIE,8f8c4ea0-7426-4188-bb6c-94fa1ecb1e33.jpg,7,"[[[tensor(-0.2863), tensor(-0.6980), tensor(-0...",tensor(4)
4142,38513,BORDER COLLIE,cce5499f-eb04-4d8c-8d3f-c9c9bee16c5c.jpg,8,"[[[tensor(0.5186), tensor(0.7422), tensor(0.20...",tensor(4)


In [ ]:
#create dataframe for loader
df = pd.concat([data_csv['data'], data_csv['class']], ignore_index=True, axis = 1)
df

In [ ]:
# transform to campatible datatype
df_dict = df.to_dict('index')
print(type(df_dict))
df_dict

# Train - Test Split and Dataloader

In [ ]:
# train, test split
train, test = train_test_split(df_dict, test_size = 0.25, random_state=42, shuffle = True)
train

In [ ]:
#dataloader
train_loader = torch.utils.data.DataLoader(train, batch_size= 2, shuffle=True)
test_loader = torch.utils.data.DataLoader(test, batch_size= 1, shuffle=True)

# Define cnn model

In [ ]:
#Define model

class Net(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
        nn.Conv2d(3, 6, (3,3)),
        nn.ReLU(),
        nn.Conv2d(6, 12, (3,3), padding=1),
        nn.ReLU(),
        nn.Flatten(),
        nn.Linear(8112, 5),
        )

    def forward(self, x):
        return self.model(x)


model = Net()
print(model)

In [ ]:
# loss function and optimezer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training

In [ ]:
max_epochs = 10
train_losses = []
for epoch in range(max_epochs): 
    train_loss = 0
    for i, data in enumerate(train_loader):
        
        inputs = data[0] 
        labels = data[1]
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * inputs.size(0)
        
    train_loss = train_loss/len(train_loader.sampler)
    print(f'Epoch: {epoch + 1}/{max_epochs} Loss: {train_loss :.5f}')
    train_losses.append(train_loss)


print('Finished Training')

# Evaluating

In [ ]:
#eval
with torch.no_grad():
    correct = 0
    total = 0
    for data in test_loader:
        
        inputs = data[0] 
        labels = data[1]
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
          
    print('Test Accuracy of the model: {} %'.format(100 * correct / total))

# Plotting

In [ ]:
#visualize
plt.plot(train_losses, label='Training loss')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend(frameon=False)

In [18]:
#plot muss noch angepasst werden
plt.rcParams["savefig.bbox"] = 'tight'
orig_img = df[42]
# if you change the seed, make sure that the randomly-applied transforms
# properly show that the image can be both transformed and *not* transformed!
torch.manual_seed(0)


def plot(imgs, with_orig=True, row_title=None, **imshow_kwargs):
    if not isinstance(imgs[0], list):
        # Make a 2d grid even if there's just 1 row
        imgs = [imgs]

    num_rows = len(imgs)
    num_cols = len(imgs[0]) + with_orig
    fig, axs = plt.subplots(nrows=num_rows, ncols=num_cols, squeeze=False)
    for row_idx, row in enumerate(imgs):
        row = [orig_img] + row if with_orig else row
        for col_idx, img in enumerate(row):
            ax = axs[row_idx, col_idx]
            ax.imshow(np.asarray(img), **imshow_kwargs)
            ax.set(xticklabels=[], yticklabels=[], xticks=[], yticks=[])

    if with_orig:
        axs[0, 0].set(title='Original image')
        axs[0, 0].title.set_size(8)
    if row_title is not None:
        for row_idx in range(num_rows):
            axs[row_idx, 0].set(ylabel=row_title[row_idx])

    plt.tight_layout()

KeyError: 42